# Notebook Interactivo: API REST para Migración de CSV a Base de Datos SQL

Este notebook te permite explorar y ejecutar paso a paso cada componente del proyecto de API REST para migración de datos CSV a una base de datos SQL.

## Estructura del Proyecto

```
csv_api_migration/
│
├── app/
│   ├── database/
│   │   ├── create_db.py       # Script para crear la base de datos
│   │   ├── db_manager.py      # Gestor de operaciones de base de datos
│   │   └── migration.db       # Archivo de base de datos SQLite (generado automáticamente)
│   │
│   ├── routes/                # Directorio para rutas adicionales (extensible)
│   │
│   ├── utils/
│   │   └── csv_processor.py   # Utilidades para procesar archivos CSV
│   │
│   └── main.py               # Punto de entrada principal de la API
│
├── data/                     # Directorio para archivos CSV
│
├── tests/                    # Directorio para pruebas unitarias
│
├── test_api.py               # Script para probar la API
│
├── requirements.txt          # Dependencias del proyecto
│
└── README.md                 # Documentación del proyecto
```

## Paso 1: Configuración del Entorno

Primero, instalamos las dependencias necesarias:

In [1]:
!pip install fastapi uvicorn python-multipart requests


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\VladimirPerez\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
!pip install pandas


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\VladimirPerez\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
!pip install python-multipart==0.0.6


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\VladimirPerez\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
!pip check

chromadb 0.6.3 has requirement fastapi>=0.95.2, but you have fastapi 0.95.1.
langchain 0.3.19 has requirement pydantic<3.0.0,>=2.7.4, but you have pydantic 1.10.22.
langchain-community 0.0.38 has requirement langchain-core<0.2.0,>=0.1.52, but you have langchain-core 0.3.35.
langchain-core 0.3.35 has requirement pydantic<3.0.0,>=2.5.2; python_full_version < "3.12.4", but you have pydantic 1.10.22.


## Paso 2: Análisis de los Archivos CSV

Vamos a examinar la estructura de los archivos CSV para entender los datos que vamos a migrar:

In [9]:
import pandas as pd
import os

# Definir la ruta a los archivos CSV
data_dir = os.path.join(os.getcwd(), 'data')

# Verificar si los archivos existen
print(f"Archivos en el directorio de datos: {os.listdir(data_dir)}")

# Leer los archivos CSV
departments_df = pd.read_csv(os.path.join(data_dir, 'departments.csv'), header=None, names=['id', 'department'])
jobs_df = pd.read_csv(os.path.join(data_dir, 'jobs.csv'), header=None, names=['id', 'job'])
employees_df = pd.read_csv(os.path.join(data_dir, 'hired_employees.csv'), header=None, 
                          names=['id', 'name', 'datetime', 'department_id', 'job_id'])

# Mostrar las primeras filas de cada DataFrame
print("\nDepartments:")
print(departments_df.head())

print("\nJobs:")
print(jobs_df.head())

print("\nHired Employees:")
print(employees_df.head())

# Mostrar información sobre los DataFrames
print("\nInformación sobre Departments:")
print(f"Número de registros: {len(departments_df)}")
print(departments_df.dtypes)

print("\nInformación sobre Jobs:")
print(f"Número de registros: {len(jobs_df)}")
print(jobs_df.dtypes)

print("\nInformación sobre Hired Employees:")
print(f"Número de registros: {len(employees_df)}")
print(employees_df.dtypes)
print(f"Valores nulos en department_id: {employees_df['department_id'].isna().sum()}")
print(f"Valores nulos en job_id: {employees_df['job_id'].isna().sum()}")

Archivos en el directorio de datos: ['departments.csv', 'hired_employees.csv', 'jobs.csv']

Departments:
   id                department
0   1        Product Management
1   2                     Sales
2   3  Research and Development
3   4      Business Development
4   5               Engineering

Jobs:
   id                        job
0   1        Marketing Assistant
1   2                   VP Sales
2   3         Biostatistician IV
3   4  Account Representative II
4   5               VP Marketing

Hired Employees:
   id            name              datetime  department_id  job_id
0   1     Harold Vogt  2021-11-07T02:48:42Z            2.0    96.0
1   2        Ty Hofer  2021-05-30T05:43:46Z            8.0     NaN
2   3     Lyman Hadye  2021-09-01T23:27:38Z            5.0    52.0
3   4   Lotti Crowthe  2021-10-01T13:04:21Z           12.0    71.0
4   5  Gretna Lording  2021-10-10T22:22:17Z            6.0    80.0

Información sobre Departments:
Número de registros: 12
id             int64
d

## Paso 3: Creación de la Base de Datos

Ahora vamos a examinar y ejecutar el script para crear la base de datos SQLite:

In [ ]:
# Primero, veamos el contenido del script create_db.py
with open('app/database/create_db.py', 'r') as file:
    print(file.read())

In [ ]:
# Ahora, importamos y ejecutamos la función para crear la base de datos
import sys
sys.path.append(os.getcwd())

from app.database.create_db import create_database

db_path = create_database()
print(f"Base de datos creada en: {db_path}")

## Paso 4: Gestor de Base de Datos

Examinemos el gestor de base de datos que se encarga de las operaciones con SQLite:

In [4]:
# Ver el contenido del script db_manager.py
with open('app/database/db_manager.py', 'r') as file:
    print(file.read())

"""
MÃ³dulo para manejar las operaciones de base de datos
"""
import sqlite3
import os
from typing import List, Dict, Any, Tuple

class DatabaseManager:
    def __init__(self, db_path=None):
        """
        Inicializa el gestor de base de datos.
        
        Args:
            db_path: Ruta al archivo de base de datos SQLite. Si es None, 
                    se usa la ruta predeterminada.
        """
        if db_path is None:
            db_dir = os.path.dirname(os.path.abspath(__file__))
            self.db_path = os.path.join(db_dir, 'migration.db')
        else:
            self.db_path = db_path
    
    def get_connection(self) -> Tuple[sqlite3.Connection, sqlite3.Cursor]:
        """
        Obtiene una conexiÃ³n a la base de datos.
        
        Returns:
            Tupla con la conexiÃ³n y el cursor.
        """
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        return conn, cursor
    
    def close_connection(self, conn: sqlite3.Co

In [5]:
# Probar el gestor de base de datos con una inserción simple
from app.database.db_manager import DatabaseManager

# Crear una instancia del gestor de base de datos
db_manager = DatabaseManager()

# Insertar un departamento de prueba
test_department = [{"id": 100, "department": "Test Department"}]
inserted_count = db_manager.insert_batch("departments", test_department)
print(f"Registros insertados: {inserted_count}")

# Verificar la inserción
result = db_manager.execute_query("SELECT * FROM departments WHERE id = 100")
print(f"Resultado de la consulta: {result}")

Registros insertados: 1
Resultado de la consulta: [(100, 'Test Department')]


## Paso 5: Procesador de CSV

Veamos cómo funciona el procesador de archivos CSV:

In [6]:
# Ver el contenido del script csv_processor.py
with open('app/utils/csv_processor.py', 'r') as file:
    print(file.read())

"""
Utilidades para procesar archivos CSV
"""
import csv
import os
from typing import List, Dict, Any

def parse_csv_file(file_path: str) -> List[Dict[str, Any]]:
    """
    Lee un archivo CSV y lo convierte en una lista de diccionarios.
    
    Args:
        file_path: Ruta al archivo CSV.
        
    Returns:
        Lista de diccionarios con los datos del CSV.
    """
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"El archivo {file_path} no existe")
    
    data = []
    
    # Detectar el tipo de archivo por su nombre
    file_name = os.path.basename(file_path).lower()
    
    if 'department' in file_name:
        # Estructura para departments.csv: id, department
        with open(file_path, 'r', encoding='utf-8') as file:
            csv_reader = csv.reader(file)
            for row in csv_reader:
                if len(row) >= 2:
                    data.append({
                        'id': int(row[0]),
                        'department': row[1]
 

In [7]:
# Probar el procesador de CSV con uno de los archivos
from app.utils.csv_processor import parse_csv_file, validate_batch_size

# Procesar el archivo departments.csv
departments_data = parse_csv_file(os.path.join(data_dir, 'departments.csv'))
print(f"Número de registros procesados: {len(departments_data)}")
print("Primeros 5 registros:")
for i, record in enumerate(departments_data[:5]):
    print(f"  {i+1}. {record}")

# Validar el tamaño del lote
is_valid = validate_batch_size(departments_data)
print(f"¿El tamaño del lote es válido? {is_valid}")

Número de registros procesados: 12
Primeros 5 registros:
  1. {'id': 1, 'department': 'Product Management'}
  2. {'id': 2, 'department': 'Sales'}
  3. {'id': 3, 'department': 'Research and Development'}
  4. {'id': 4, 'department': 'Business Development'}
  5. {'id': 5, 'department': 'Engineering'}
¿El tamaño del lote es válido? True


## Paso 6: API REST Principal

Examinemos el archivo principal de la API REST:

In [8]:
# Ver el contenido del script main.py
with open('app/main.py', 'r') as file:
    print(file.read())

"""
ConfiguraciÃ³n principal de la API REST
"""
from fastapi import FastAPI, UploadFile, File, HTTPException, Body
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
import os
import tempfile
import shutil
from typing import List, Dict, Any

# Importar mÃ³dulos propios
from app.database.create_db import create_database
from app.database.db_manager import DatabaseManager
from app.utils.csv_processor import parse_csv_file, validate_batch_size

# Crear la aplicaciÃ³n FastAPI
app = FastAPI(
    title="API de MigraciÃ³n CSV",
    description="API REST para migrar datos desde archivos CSV a una base de datos SQL",
    version="1.0.0"
)

# Configurar CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Inicializar la base de datos al iniciar la aplicaciÃ³n
@app.on_event("startup")
async def startup_event():
    db_path = create_database()
    print(f"B

## Paso 7: Iniciar la API REST

Para iniciar la API REST, necesitamos ejecutar el siguiente comando en una terminal:

```bash
uvicorn app.main:app --host 127.0.0.1 --port 8080
```

Esto iniciará el servidor en http://localhost:8080.

**Nota**: No podemos ejecutar este comando directamente en el notebook porque bloquearía la ejecución de las siguientes celdas. En su lugar, puedes abrir una terminal separada y ejecutar el comando allí.

## Paso 8: Pruebas de la API

Veamos el script de pruebas de la API:

In [4]:
# Ver el contenido del script test_api.py
with open('test_api.py', 'r') as file:
    print(file.read())

"""
Script para probar la API REST
"""
import os
import requests
import json

# ConfiguraciÃ³n
API_URL = "http://localhost:8080"
DATA_DIR = os.path.join(os.path.dirname(os.path.abspath(__file__)), "data")

def test_api_status():
    """Prueba el estado de la API"""
    print("\n=== Probando estado de la API ===")
    response = requests.get(f"{API_URL}/")
    print(f"Respuesta: {response.status_code}")
    print(json.dumps(response.json(), indent=2))
    return response.status_code == 200

def test_upload_csv(table_name, file_path):
    """Prueba la carga de un archivo CSV"""
    print(f"\n=== Probando carga de CSV para {table_name} ===")
    with open(file_path, 'rb') as file:
        files = {'file': (os.path.basename(file_path), file, 'text/csv')}
        response = requests.post(f"{API_URL}/upload/{table_name}", files=files)
    
    print(f"Respuesta: {response.status_code}")
    print(json.dumps(response.json(), indent=2))
    return response.status_code == 201

def test_batch_in

Para ejecutar las pruebas, necesitas tener la API en ejecución en otra terminal. Luego, puedes ejecutar el siguiente comando:

```bash
python test_api.py
```

O puedes ejecutar las pruebas manualmente, como se muestra a continuación:

In [6]:
# Prueba manual del estado de la API (asegúrate de que la API esté en ejecución)
import requests
import json

try:
    response = requests.get("http://localhost:8080/")
    print(f"Respuesta: {response.status_code}")
    print(json.dumps(response.json(), indent=2))
except requests.exceptions.ConnectionError:
    print("Error: No se pudo conectar a la API. Asegúrate de que esté en ejecución.")

Respuesta: 200
{
  "message": "API de Migraci\u00f3n CSV activa",
  "status": "OK"
}


## Paso 9: Prueba de Carga de CSV

Si la API está en ejecución, podemos probar la carga de un archivo CSV:

In [14]:
# Prueba de carga de CSV (asegúrate de que la API esté en ejecución)
import requests

try:
    # Cargar el archivo departments.csv
    departments_csv = os.path.join(data_dir, 'departments.csv')
    
    with open(departments_csv, 'rb') as file:
        files = {'file': (os.path.basename(departments_csv), file, 'text/csv')}
        response = requests.post("http://localhost:8080/upload-from-path/departments", 
                                 json={"file_path": departments_csv},
                                 headers={"Content-Type": "application/json"}
                                )
    
    print(f"Respuesta: {response.status_code}")
    print(json.dumps(response.json(), indent=2))
except requests.exceptions.ConnectionError:
    print("Error: No se pudo conectar a la API. Asegúrate de que esté en ejecución.")

Respuesta: 500
{
  "detail": "UNIQUE constraint failed: departments.id"
}


## Paso 10: Prueba de Inserción por Lotes

Si la API está en ejecución, podemos probar la inserción de un lote de registros:

In [12]:
# Prueba de inserción por lotes (asegúrate de que la API esté en ejecución)
import requests

try:
    # Datos de prueba
    departments_batch = [
        {"id": 101, "department": "Quality Assurance"},
        {"id": 102, "department": "Customer Support"}
    ]
    
    response = requests.post(
        "http://localhost:8080/batch/departments",
        json=departments_batch,
        headers={"Content-Type": "application/json"}
    )
    
    print(f"Respuesta: {response.status_code}")
    print(json.dumps(response.json(), indent=2))
except requests.exceptions.ConnectionError:
    print("Error: No se pudo conectar a la API. Asegúrate de que esté en ejecución.")

Respuesta: 201
{
  "message": "Lote insertado exitosamente en la tabla departments",
  "records_inserted": 2
}


## Paso 11: Verificación de Datos en la Base de Datos

Podemos verificar que los datos se hayan insertado correctamente en la base de datos:

In [15]:
# Verificar los datos en la base de datos
from app.database.db_manager import DatabaseManager

# Crear una instancia del gestor de base de datos
db_manager = DatabaseManager()

# Consultar los departamentos
departments = db_manager.execute_query("SELECT * FROM departments")
print(f"Número de departamentos: {len(departments)}")
print("Primeros 10 departamentos:")
for i, dept in enumerate(departments[:10]):
    print(f"  {i+1}. ID: {dept[0]}, Nombre: {dept[1]}")

# Consultar los trabajos
jobs = db_manager.execute_query("SELECT * FROM jobs")
print(f"\nNúmero de trabajos: {len(jobs)}")
print("Primeros 10 trabajos:")
for i, job in enumerate(jobs[:10]):
    print(f"  {i+1}. ID: {job[0]}, Nombre: {job[1]}")

# Consultar los empleados
employees = db_manager.execute_query("SELECT * FROM hired_employees")
print(f"\nNúmero de empleados: {len(employees)}")
print("Primeros 10 empleados:")
for i, emp in enumerate(employees[:10]):
    print(f"  {i+1}. ID: {emp[0]}, Nombre: {emp[1]}, Fecha: {emp[2]}, Depto: {emp[3]}, Trabajo: {emp[4]}")

Número de departamentos: 5
Primeros 10 departamentos:
  1. ID: 12, Nombre: Quality Assurance
  2. ID: 13, Nombre: Customer Support
  3. ID: 100, Nombre: Test Department
  4. ID: 101, Nombre: Quality Assurance
  5. ID: 102, Nombre: Customer Support

Número de trabajos: 2
Primeros 10 trabajos:
  1. ID: 183, Nombre: Data Scientist
  2. ID: 184, Nombre: DevOps Engineer

Número de empleados: 2
Primeros 10 empleados:
  1. ID: 2000, Nombre: John Doe, Fecha: 2022-01-01T10:00:00Z, Depto: 5, Trabajo: 10
  2. ID: 2001, Nombre: Jane Smith, Fecha: 2022-01-02T11:30:00Z, Depto: 3, Trabajo: 15


## Resumen

En este notebook, hemos explorado y ejecutado paso a paso cada componente del proyecto de API REST para migración de datos CSV a una base de datos SQL:

1. Analizamos la estructura de los archivos CSV
2. Creamos la base de datos SQLite
3. Probamos el gestor de base de datos
4. Probamos el procesador de archivos CSV
5. Examinamos la API REST principal
6. Probamos la API (si estaba en ejecución)
7. Verificamos los datos en la base de datos

Para ejecutar la API completa, recuerda que debes ejecutar el siguiente comando en una terminal:

```bash
uvicorn app.main:app --reload
```

Y para probar la API, puedes ejecutar:

```bash
python test_api.py
```

O acceder a la documentación interactiva en:
- Swagger UI: http://localhost:8000/docs
- ReDoc: http://localhost:8000/redoc